Student Name: Ahmad Rashidh Ameer Hamsha

Student ID: 23264306

In [13]:
import pandas as pd
import numpy as np

In [14]:
!wget https://norvig.com/ngrams/count_1w.txt
!wget https://norvig.com/ngrams/count_2w.txt

--2023-10-29 19:57:34--  https://norvig.com/ngrams/count_1w.txt
Resolving norvig.com (norvig.com)... 158.106.138.13
Connecting to norvig.com (norvig.com)|158.106.138.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4956241 (4.7M) [text/plain]
Saving to: ‘count_1w.txt.1’

count_1w.txt.1      100%[===================>]   4.73M  --.-KB/s    in 0.1s    

2023-10-29 19:57:35 (32.2 MB/s) - ‘count_1w.txt.1’ saved [4956241/4956241]

--2023-10-29 19:57:35--  https://norvig.com/ngrams/count_2w.txt
Resolving norvig.com (norvig.com)... 158.106.138.13
Connecting to norvig.com (norvig.com)|158.106.138.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5566017 (5.3M) [text/plain]
Saving to: ‘count_2w.txt.1’

count_2w.txt.1      100%[===================>]   5.31M  --.-KB/s    in 0.1s    

2023-10-29 19:57:35 (42.1 MB/s) - ‘count_2w.txt.1’ saved [5566017/5566017]



In [27]:
unigram_df = pd.read_csv('count_1w.txt', delimiter='\t')
unigram_df.columns = ['unigram','count']


bigram_df = pd.read_csv('count_2w.txt', delimiter='\t')
bigram_df.columns = ['bigram','count']
totalUnigrams = unigram_df.shape[0]
totalBigrams = bigram_df.shape[0]

print(f'Number of unigrams: {totalUnigrams} Number of bigrams: {totalBigrams} ')

Number of unigrams: 333332 Number of bigrams: 286357 


In [28]:
display(unigram_df.head(100))
display(bigram_df.head(100))

,unigram,count
0,of,13151942776
1,and,12997637966
2,to,12136980858
3,a,9081174698
4,in,8469404971
...,...,...
95,service,519537222
96,x,508609523
97,than,502609275
98,find,502043038


,bigram,count
0,0km to,116103
1,1000s of,939476
2,100s of,539389
3,100th anniversary,158621
4,10am to,376141
...,...,...
95,25th anniversary,261023
96,25th of,397735
97,26th of,271707
98,27th of,276619


## Task 1. a
Finding the probability without 1 smoothing

### $P(w_{i}|w_{i-1}) = \frac{c(w_{i-1},w_{i})} {c(w_{i-1})}$  
```
where, c() -> count of,  
       w -> represents the word,  
       i -> current index  
```

In [29]:
def probability(sentence):
  sentenceSplit = sentence.split()
  print(f"sentence: {sentenceSplit}")
  sentenceProbability = 1


  for idx in range(len(sentenceSplit) - 1):
    word = sentenceSplit[idx]
    biword = ' '.join(sentenceSplit[idx:idx+2])

    unigramCounts = unigram_df.loc[unigram_df['unigram'] == word]['count'].values
    bigramCounts = bigram_df.loc[bigram_df['bigram'] == biword]['count'].values

    if len(unigramCounts) > 0 and len(bigramCounts) > 0:
      print(f'unigram count for "{word}": {unigramCounts[0]}')
      print(f'bigram count for "{biword}": {bigramCounts}')
      bigramProbability = bigramCounts[0] / unigramCounts[0]
      print(f'bigramProbability for "{biword}": {bigramProbability}')
      sentenceProbability *= bigramProbability
    else:
      sentenceProbability = 0

    print("\n--------------------------------------------------\n")

  print(f'sentence probability for "{sentence}": {sentenceProbability}')
  return sentenceProbability

In [30]:
print(probability("i love you") > probability("i hate you"))

sentence: ['i', 'love', 'you']
unigram count for "i": 3086225277
bigram count for "i love": [3979312]
bigramProbability for "i love": 0.001289378332053626

--------------------------------------------------

unigram count for "love": 201063526
bigram count for "love you": [5428714]
bigramProbability for "love you": 0.02699999402178991

--------------------------------------------------

sentence probability for "i love you": 3.481320725727335e-05
sentence: ['i', 'hate', 'you']
unigram count for "i": 3086225277
bigram count for "i hate": [876611]
bigramProbability for "i hate": 0.0002840398614232463

--------------------------------------------------

unigram count for "hate": 21274675
bigram count for "hate you": [504048]
bigramProbability for "hate you": 0.023692394830943365

--------------------------------------------------

sentence probability for "i hate you": 6.7295845445659906e-06
True


## Task 1.b
Applying add-one smoothing. We need to add the total word types to the denominator for this to do this.  

### $P^{*}(w_{n}|w_{n-1}) = \frac{c(w_{n-1}, w_{n})+1} {c(w_{n-1})+V}$

In [31]:
def probability_addone_smoothing(sentence):
  sentenceSplit = sentence.split()
  print(f"sentence: {sentenceSplit}")
  sentenceProbability = 1


  for idx in range(len(sentenceSplit) - 1):
    word = sentenceSplit[idx]
    biword = ' '.join(sentenceSplit[idx:idx+2])

    unigramCounts = unigram_df.loc[unigram_df['unigram'] == word]['count'].values
    bigramCounts = bigram_df.loc[bigram_df['bigram'] == biword]['count'].values

    if len(unigramCounts) > 0 and len(bigramCounts) > 0:
      print(f'unigram count for "{word}": {unigramCounts[0]}')
      print(f'bigram count for "{biword}": {bigramCounts}')
      bigramProbability = (bigramCounts[0] + 1) / (unigramCounts[0] + totalUnigrams)
      print(f'bigramProbability for "{biword}": {bigramProbability}')
      sentenceProbability *= bigramProbability
    else:
      sentenceProbability = 0

    print("\n--------------------------------------------------\n")

  print(f'sentence probability for "{sentence}": {sentenceProbability}')
  return sentenceProbability

In [34]:
print(probability_addone_smoothing("i love you") > probability_addone_smoothing("i hate you"))

sentence: ['i', 'love', 'you']
unigram count for "i": 3086225277
bigram count for "i love": [3979312]
bigramProbability for "i love": 0.0012892394100007837

--------------------------------------------------

unigram count for "love": 201063526
bigram count for "love you": [5428714]
bigramProbability for "love you": 0.026955311288917923

--------------------------------------------------

sentence probability for "i love you": 3.4751849622512007e-05
sentence: ['i', 'hate', 'you']
unigram count for "i": 3086225277
bigram count for "i hate": [876611]
bigramProbability for "i hate": 0.0002840095106063803

--------------------------------------------------

unigram count for "hate": 21274675
bigram count for "hate you": [504048]
bigramProbability for "hate you": 0.02332695467934641

--------------------------------------------------

sentence probability for "i hate you": 6.625076982418387e-06
True


### Task 2 - Shannon Visualization

In [33]:
def ShannonVisualization(seed="<S>"):

    sentence = seed
    current_word = seed
    # padding to form a step for better visualization
    padding = ""

    breaking_condition = 10

    # loop breaks on three instance - whatever comes first
    # + random value of breaking condition is below 0.10
    # + if there is no next word found
    # + if <S> is found
    while breaking_condition > 0.10:

        current_word_bigram = bigram_df[bigram_df['bigram'].str.startswith(current_word + " ")]
        probability = current_word_bigram['count'] / current_word_bigram['count'].sum()

        next_word_list = current_word_bigram['bigram'].apply(lambda x: x.split()[1])
        if len(next_word_list) == 0:
          break

        next_word = np.random.choice(next_word_list, p=probability.to_numpy())
        sentence += f" {next_word}"

        print(f"{padding} {current_word} {next_word}")
        padding += " "

        current_word = next_word
        if next_word == '<S>':
          break

        breaking_condition = np.random.uniform(0, 1)

    return sentence

ShannonVisualization()

 <S> it
  it lacked
   lacked the
    the executive
     executive secretary


'<S> it lacked the executive secretary'